In [1]:
import requests
from bs4 import BeautifulSoup
import requests 
import json
import os
from dotenv import load_dotenv
import pandas as pd
from pandas import json_normalize
import tweepy
import time
load_dotenv()

True

In [2]:
apikey = os.getenv("apikey") #apikey de lastfm

# Spotify Most streamed
- scrapping

[charset: most streamed albums](https://chartmasters.org/spotify-most-streamed-albums/?y=alltime)
Last updated on 11/06/21

In [3]:
url_spotify = "https://chartmasters.org/spotify-most-streamed-albums/?y=alltime"
html_spotify = requests.get(url_spotify)
soup_spotify = BeautifulSoup(html_spotify.content,"html.parser")

KeyboardInterrupt: 

In [ ]:
tablas_spotify = soup_spotify.findAll("table")

In [ ]:
spotify = tablas_spotify[1]
spotify.find_all("tr") #cuerpo
spotify.find_all("th") #cabeceros

In [ ]:
def data_spotify(element):
    Top_alb = []
    for s in element.find_all("tr"):
        fila = [elemento for elemento in s.find_all("td")]
        if len(fila)>1:
            spot_dicc = {"Rank" : int(fila[0].text),
                        "Artist": fila[2].text.strip(),
                         "Album": fila[3].text.strip(),
                         "Total" :int(fila[4].text.replace(",","")),
                         "Daily" : int(fila[5].text.replace(",",""))
                        }
            Top_alb.append(spot_dicc)
    return Top_alb

In [ ]:
element = spotify
Top_alb_spotify = data_spotify(element)
Top_alb_spotify[1]

In [ ]:
Spotify_500 = pd.DataFrame(Top_alb_spotify)
Spotify_500

In [ ]:
Spotify_500.to_csv("../data/Spotify_500.csv")

# LastFM 50 Top Artists

In [ ]:
url_last50 = f"http://ws.audioscrobbler.com/2.0/?method=chart.gettopartists&api_key={apikey}&format=json"
req_last50 = requests.get(url_last50).json()

In [ ]:
last_50 = req_last50["artists"]["artist"]
last_top50 = pd.DataFrame.from_dict(last_50)

In [ ]:
last_top50.sample(5)

In [ ]:
last_top50.to_csv("../data/last_top50_image.csv") #me guardo esto por si acso

In [ ]:
top50_lastfm = last_top50[['name', 'playcount', 'listeners']]

In [ ]:
top50_lastfm.sample(5)

In [ ]:
top50_lastfm.to_csv("../data/top50_lastfm.csv")

# Top Sells

[top50]("https://www.businessinsider.com/50-best-selling-albums-all-time-2016-9")
Travis Clark Updated Sep 22, 2021, 8:18 PM

In [4]:
url_insider = "https://www.businessinsider.com/50-best-selling-albums-all-time-2016-9"
html_insider = requests.get(url_insider)
soup_insider = BeautifulSoup(html_insider.content,"html.parser")

In [5]:
tags_insider = soup_insider.find_all("div", {"class": "slide-layout"})
tags_insider[1]

<div class="slide-layout clearfix">
<div class="slide-title clearfix" data-e2e-name="slide-title">
<h2 class="slide-title-text">49. Kenny Rogers — "Kenny Rogers' Greatest Hits"</h2>
</div>
<figure class="figure image-figure-image" data-e2e-name="image-figure-image" data-media-container="image" data-type="img">
<div class="lazy-holder" style="padding-top: calc(100% * 898 / 980)">
<img alt="kenny rogers" class="lazy-image" data-content-type="image/jpeg" data-srcs='{"https://i.insider.com/57e43003077dccfc298b4a77":{"contentType":"image/jpeg","aspectRatioW":490,"aspectRatioH":449}}' src="data:image/svg+xml,%3Csvg xmlns='http://www.w3.org/2000/svg' viewBox='0 0 1 1'%3E%3C/svg%3E"/>
<noscript>
<img src="https://i.insider.com/57e43003077dccfc298b4a77?width=600&amp;format=jpeg&amp;auto=webp"/>
</noscript>
</div>
<span class="image-source-only">
<span class="image-source headline-regular" data-e2e-name="image-source">
                                                  Liberty
                   

In [6]:
tags_insider[0].find("h2").getText()

'50. Led Zeppelin —\xa0"Led Zeppelin II"'

In [7]:
tags_insider[0].find_all("p")[0].getText()

'Certified units: 12\xa0million'

In [8]:
def data_sells(element):
    lista = []
    for t in element:
        dicc_t = {"album": t.find("h2").getText(),
                 "certified units" : t.find("p").getText()}
        lista.append(dicc_t)
    return lista

In [9]:
insider_data = data_sells(tags_insider)
insider_data[0]

{'album': '50. Led Zeppelin —\xa0"Led Zeppelin II"',
 'certified units': 'Certified units: 12\xa0million'}

In [10]:
insider = pd.DataFrame(insider_data)
insider.sample(3)

,album,certified units
28,"22. Metallica — ""Metallica""",Certified units: 16 million
43,"7. Led Zeppelin — ""Led Zeppelin IV""",Certified units: 23 million
37,"13. Garth Brooks — ""No Fences""",Certified units: 18 million


In [11]:
insider["Rank"] = insider["album"].str.extract(r"(^\d+)")
insider["Millions"] = insider["certified units"].str.extract(r"(\d+)")
insider["album"] = insider["album"].str.replace(" – "," — ",regex = True)
insider["Artist"] = insider["album"].str.extract(r"(^\d+.\s(.+?)—)")[1]
insider["Title"] = insider["album"].str.extract(r"(—\s(.+?)$)")[1]
insider["Title"] = insider["Title"].str.replace('"',"")

In [12]:
insider.sample(3)

,album,certified units,Rank,Millions,Artist,Title
34,"16. Boston — ""Boston""",Certified units: 17 million,16,17,Boston,Boston
45,"5. The Beatles — ""The Beatles"" (""The White Alb...",Certified units: 24 million,5,24,The Beatles,The Beatles (The White Album)
6,"44. Boyz II Men — ""II""",Certified units: 12 million,44,12,Boyz II Men,II


In [ ]:
insider.to_csv("../data/top_sells_certified.csv")

In [13]:
insider[["Millions","Rank"]] = insider[["Millions","Rank"]].astype("int64")
top_sells = insider[["Rank","Artist","Title","Millions"]]

In [14]:
top_sells.head()

,Rank,Artist,Title,Millions
0,50,Led Zeppelin,Led Zeppelin II,12
1,49,Kenny Rogers,Kenny Rogers' Greatest Hits,12
2,48,Kenny G,Breathless,12
3,47,Jewel,Pieces of You,12
4,46,Def Leppard,Hysteria,12


In [ ]:
top_sells.to_csv("../data/top_sells.csv")

#### enriquecimiento top sells

In [15]:
def urls_llamadas(df, colartista, colalbum): #pasar nombre tabla por parametros
    """
    esta función recibe 3 parámetros:
        df = un dataframe
        colartista = la columna que se refiere al artista y compatible con url
        colalbum = la columna que se reifere al album y compatible con url
    llama a la api de last fm del album.
    hace un return de una lista de diccionarios con los datos que tiene lastfm de todos los álbumes del dataframe    
    """
    api_urls = []
    apikey = os.getenv("apikey")
    for s in range(len(df)): #cambiarnombre tabla
        artist = df.loc[s,colartista] #tambien pasar nombre columnas por parametros
        album = df.loc[s,colalbum]
        api_urls.append(f"http://ws.audioscrobbler.com/2.0/?method=album.getinfo&api_key={apikey}&artist={artist}&album={album}&format=json")
    request_dic = [] 
    i = 0
    for a in api_urls:
        res = requests.get(a).json()
        request_dic.append(res["album"])
        if i%20==0:
            print(f"{i} done")
        elif i == (len(df) -1) :
            print(f"index {i} reached, finished")
        i+=1
    return request_dic

In [16]:
df = top_sells
colartista = "Artist"
colalbum = "Title"
req_top_sales = urls_llamadas(df, colartista, colalbum)

0 done
20 done
40 done
index 49 reached, finished


In [17]:
req_top_sales[3]

{'artist': 'Jewel',
 'mbid': '0970ae4a-c35a-4c55-8aa4-bb19af6695a4',
 'tags': {'tag': [{'url': 'https://www.last.fm/tag/female+vocalists',
    'name': 'female vocalists'},
   {'url': 'https://www.last.fm/tag/folk', 'name': 'folk'},
   {'url': 'https://www.last.fm/tag/pop', 'name': 'pop'},
   {'url': 'https://www.last.fm/tag/90s', 'name': '90s'},
   {'url': 'https://www.last.fm/tag/singer-songwriter',
    'name': 'singer-songwriter'}]},
 'playcount': '2934308',
 'image': [{'size': 'small',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/34s/b5830d83842eef2b4d5bd2648066909d.png'},
  {'size': 'medium',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/64s/b5830d83842eef2b4d5bd2648066909d.png'},
  {'size': 'large',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/174s/b5830d83842eef2b4d5bd2648066909d.png'},
  {'size': 'extralarge',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/b5830d83842eef2b4d5bd2648066909d.png'},
  {'size': 'mega',
   '#text': 'https://lastfm.freetls.

In [ ]:
top_sales_df = pd.DataFrame(req_top_sales)
top_sales_df.sample(3)

In [ ]:
top_50_sales = top_sales_df[["artist","name","playcount","listeners"]]
top_50_sales.sample(3)

In [ ]:
top_50_sales_rich = top_sells.merge(top_50_sales,left_index=True, right_index=True)
top_50_sales_rich.sample(3)

In [ ]:
top_sales_rich = top_50_sales_rich[["Rank","Artist","Title","playcount","listeners"]]
top_sales_rich.sample(3)

In [ ]:
top_sales_rich.to_csv("../data/top_sells_rich.csv")

# Top 500 albums Rolling Stones enrinquecimiento con LastFM

### carga de archivo limpio

In [ ]:
stone = pd.read_csv("../data/stone.csv", encoding='latin-1')
stone = stone.drop("Unnamed: 0",axis=1)
stone.sample(3)

### llamada a las apis de lastFM

In [ ]:
def urls_llamadas ():
    api_urls = []
    apikey = os.getenv("apikey")
    for s in range(len(stone)):
        artist = stone.loc[s,'Artist']
        album = stone.loc[s,'Alb_url']
        api_urls.append(f"http://ws.audioscrobbler.com/2.0/?method=album.getinfo&api_key={apikey}&artist={artist}&album={album}&format=json")
    request_dic = [] 
    i = 0
    for a in api_urls:
        res = requests.get(a).json()
        request_dic.append(res["album"])
        if i%50==0:
            print(f"{i} done")
        elif i == 499:
            print(f"index {i} reached, finished")
        i+=1
    return request_dic




In [ ]:
Top500_last = urls_llamadas()

In [ ]:
Top500_last_df = pd.DataFrame(Top500_last)

In [ ]:
Top500_last_df.sample(3)

In [ ]:
Top500_last_df.to_csv("../data/Top500LastFM_tracks.csv") #meloguardoporsiacaso

In [ ]:
Lastfm_500 = Top500_last_df[["artist","playcount","name","listeners"]]
Lastfm_500.sample(3)

In [ ]:
stone_500 = stone[["Number","Year","Album","Artist","Gen","Subgen"]]
stone_500.sample(3)

In [ ]:
#stone_alb.merge(reqs,left_index=True, right_index=True)
stone_500_richment = stone_500.merge(Lastfm_500,left_index=True, right_index=True)
stone_500_richment.sample(3)

In [ ]:
stone_500_rich =stone_500_richment [["Number","Year","Album","Artist","Gen","Subgen","playcount","listeners"]]
stone_500_rich

In [ ]:
stone_500_rich.to_csv("../data/stone_500_rich.csv")